In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import csv

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import load_model

print(tf.__version__, np.__version__)

# Expected output 2.9.0, 1.26.4

2.9.0 1.26.4


# Detect GPU and limit memory usage

In [2]:
USE_GPU = True

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus != []:
    print(gpus)
    for gpu in gpus: 
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("No GPU on this machine")

# Expected output [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
if USE_GPU == False:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found and successfully configured')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU found and successfully configured


# Load training and testing data

In [4]:
def get_preprocessing_function(model_name):
    """Get the appropriate preprocessing function for each model"""
    preprocess_dict = {
        'DenseNet201': tf.keras.applications.densenet.preprocess_input,
        'EfficientNetB7': tf.keras.applications.efficientnet.preprocess_input,
        'EfficientNetV2L': tf.keras.applications.efficientnet_v2.preprocess_input,
        'InceptionResNetV2': tf.keras.applications.inception_resnet_v2.preprocess_input,
        'InceptionV3': tf.keras.applications.inception_v3.preprocess_input,
        'MobileNetV3Large': tf.keras.applications.mobilenet_v3.preprocess_input,
        'NASNetLarge': tf.keras.applications.nasnet.preprocess_input,
        'ResNet152': tf.keras.applications.resnet.preprocess_input,
        'ResNet152V2': tf.keras.applications.resnet_v2.preprocess_input,
        'VGG19': tf.keras.applications.vgg19.preprocess_input,
        'Xception': tf.keras.applications.xception.preprocess_input
    }
    return preprocess_dict.get(model_name)

In [5]:
def preprocess_data(data, preprocess_fn):
    """Apply preprocessing to a dataset"""
    def preprocess(x, y):
        return preprocess_fn(x), y
    
    return data.map(preprocess)

In [6]:
def load_training_data(train_data_dir, model_name, train_data_ratio=0.8, batch_size=8):
    print(f"Loading training data from {train_data_dir}...")

    train_data = tf.keras.utils.image_dataset_from_directory(
        train_data_dir,
        image_size=(224, 224),
        batch_size=batch_size
    )

    # Get preprocessing function for the model
    preprocess_fn = get_preprocessing_function(model_name)
    if preprocess_fn is not None:
        train_data = preprocess_data(train_data, preprocess_fn)

    # Split into train and validation
    train_size = int(len(train_data)*train_data_ratio)
    val_size = int(len(train_data)*(1-train_data_ratio))
    train = train_data.take(train_size)
    val = train_data.skip(train_size).take(val_size)

    # Use caching and prefetching for better performance
    train = train.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    val = val.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    return train, val

def load_test_data(test_data_dir, model_name, batch_size=8):
    print(f"Loading testing data from {test_data_dir}...")

    test_data = tf.keras.utils.image_dataset_from_directory(
        test_data_dir,
        image_size=(224, 224),
        batch_size=batch_size
    )

    # Get preprocessing function for the model
    preprocess_fn = get_preprocessing_function(model_name)
    if preprocess_fn is not None:
        test_data = preprocess_data(test_data, preprocess_fn)

    # Use caching and prefetching for better performance
    test_data = test_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    return test_data

# Build model

In [7]:
base_model_name = ['DesnseNet201']
INPUT_SHAPE = (224,224,3)

In [8]:
#all_model = ['DenseNet201', 'EfficientNetB7', 'EfficientNetV2L','InceptionResNetV2', 'InceptionV3', 'MobileNetV3Large', 'NASNetLarge', 'ResNet152', 'ResNet152V2', 'VGG19', 'Xception']

def get_base_model(model_name, input_shape):
    model_dict = {
        'DenseNet201': tf.keras.applications.DenseNet201,
        'EfficientNetB7': tf.keras.applications.EfficientNetB7,
        'EfficientNetV2L': tf.keras.applications.EfficientNetV2L,
        'InceptionResNetV2': tf.keras.applications.InceptionResNetV2,
        'InceptionV3': tf.keras.applications.InceptionV3,
        'MobileNetV3Large': tf.keras.applications.MobileNetV3Large,
        'NASNetLarge': tf.keras.applications.NASNetLarge,
        'ResNet152': tf.keras.applications.ResNet152,
        'ResNet152V2': tf.keras.applications.ResNet152V2,
        'VGG19': tf.keras.applications.VGG19,
        'Xception': tf.keras.applications.Xception,
    }
    
    if model_name not in model_dict:
        raise ValueError(f"Unsupported model: {model_name}")
    
    return model_dict[model_name](input_shape=input_shape, include_top=False, weights='imagenet')

In [9]:
def build_model(base_model, input_shape):

    base_model.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)

    print("Built model")
    pd.set_option('max_colwidth', None)
    layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
    pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

    return model

# Usage example

# input_shape = (224, 224, 3)
# base_model = get_base_model(model_name, input_shape)
# model = build_model(base_model, input_shape)

In [10]:
def get_callbacks(model_name, results_dir, models_dir, initial_lr="001"):
    log_file = os.path.join(results_dir, f"{model_name}_adaptive_lr{initial_lr}.csv")
    
    return [
        CSVLogger(log_file),
        ModelCheckpoint(
            filepath=os.path.join(models_dir, f"{model_name}_adaptive_lr{initial_lr}.h5"),
            save_weights_only=False,
            save_best_only=True,
            save_freq='epoch',
            verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',        # Monitors validation loss
            factor=0.1,               # Multiplies learning rate by 0.1 (reduces by 90%)
            patience=5,               # Waits 3 epochs before reducing
            verbose=1,                # Prints message when reducing LR
            min_delta=1e-4,          # Minimum change in loss to be considered an improvement
            min_lr=1e-4              # Won't reduce LR below this value
        )
    ]

In [11]:
def plot_training_history(history, model_name, results_dir, lr_suffix="001"):
    plt.figure()
    plt.plot(history.history['loss'], color='teal', label='loss')
    plt.plot(history.history['val_loss'], color='orange', label='val_loss')
    plt.title(f'{model_name} Loss (LR: 0.{lr_suffix})')
    plt.legend(loc="upper left")
    plt.savefig(os.path.join(results_dir, f"{model_name}_loss_lr{lr_suffix}.png"))
    plt.close()

# Training

In [12]:
def test_model(model_name, test_data, models_dir, results_dir):
    """
    Test model and save metrics to CSV
    
    Parameters:
        model_name: Name of the model
        test_data: Test dataset
        models_dir: Directory containing model files
        results_dir: Directory to save results
    """
    # Load model
    model = tf.keras.models.load_model(os.path.join(models_dir, f"{model_name}_adaptive_lr001.h5"))
    
    # Initialize metrics
    precision = Precision()
    recall = Recall()  # Same as Sensitivity
    accuracy = BinaryAccuracy()
    
    # Lists to store predictions and true labels for specificity calculation
    y_true = []
    y_pred = []
    
    # Evaluate model on test data
    for batch in test_data.as_numpy_iterator():
        X, y = batch
        yhat = model.predict(X)
        
        # Update metrics
        precision.update_state(y, yhat)
        recall.update_state(y, yhat)
        accuracy.update_state(y, yhat)
        
        # Store predictions and true labels
        y_true.extend(y)
        y_pred.extend([1 if pred >= 0.5 else 0 for pred in yhat])
    
    # Calculate metrics
    precision_value = float(precision.result())
    recall_value = float(recall.result())  # Sensitivity
    accuracy_value = float(accuracy.result())
    
    # Calculate F1 Score
    f1_score = 2 * (precision_value * recall_value) / (precision_value + recall_value)
    
    # Calculate Specificity
    # True Negatives / (True Negatives + False Positives)
    tn = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 0)
    fp = sum(1 for t, p in zip(y_true, y_pred) if t == 0 and p == 1)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # Create metrics dictionary
    metrics = {
        'Model': model_name,
        'Accuracy': accuracy_value,
        'Precision': precision_value,
        'Recall (Sensitivity)': recall_value,
        'Specificity': specificity,
        'F1 Score': f1_score
    }
    
    # Print results
    print(f"\n{model_name} Test Results:")
    for metric, value in metrics.items():
        if metric != 'Model':
            print(f"{metric}: {value:.4f}")
    
    # Save metrics to CSV
    csv_file = os.path.join(results_dir, 'model_metrics.csv')
    
    # Check if file exists to determine if we need to write headers
    file_exists = os.path.exists(csv_file)
    
    # Write to CSV
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=metrics.keys())
        
        # Write headers if file doesn't exist
        if not file_exists:
            writer.writeheader()
        
        writer.writerow(metrics)
    
    print(f"\nMetrics saved to: {csv_file}")
    
    # Return metrics dictionary
    return metrics

In [13]:
def train_models(model_names, train_data_dir, test_data_dir, results_base_dir, models_base_dir, max_epochs=100, batch_size=32):

    for model_name in model_names:
        print(f"Training {model_name}...")

        train, val = load_training_data(train_data_dir, model_name, train_data_ratio=0.8, batch_size=batch_size)

        # Set up model-specific directories
        results_dir = os.path.join(results_base_dir, model_name)
        models_dir = os.path.join(models_base_dir, model_name)
        os.makedirs(results_dir, exist_ok=True)
        os.makedirs(models_dir, exist_ok=True)

        # Build model
        input_shape = (224, 224, 3)
        base_model = get_base_model(model_name, input_shape)
        model = build_model(base_model, input_shape)

        # Compile model with initial learning rate
        initial_lr = 1e-2
        model.compile(
            loss=keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer=keras.optimizers.Adam(learning_rate=initial_lr),
            metrics=[keras.metrics.BinaryAccuracy()]
        )

        # Print model summary
        model.summary()
        
        # Set up callbacks with adaptive learning rate and early stopping
        callbacks = get_callbacks(model_name, results_dir, models_dir)

        # Train model
        history = model.fit(
            train,
            validation_data=val,
            epochs=max_epochs,
            verbose=1,
            callbacks=callbacks
        )

        # Plot training history
        plot_training_history(history, model_name, results_dir)

        # Test model
        test_data = load_test_data(test_data_dir, model_name, batch_size=1)
        test_model(model_name, test_data, models_dir, results_dir)

# Main

In [ ]:
model_to_train =  ['ResNet152', 'ResNet152V2', 'VGG19', 'Xception']
# input shape error 'NASNetLarge' need input shape of exact dimensions (331, 331, 3)
train_data_dir = r"D:\Kananat\TF_TMJOA_jpg_x_5px"
test_data_dir = r"D:\Kananat\TF_TMJOA_jpg_x_5px_test"

results_base_dir = r"D:\Kananat\_result\lr scheduler exp\logs"
models_base_dir = r"D:\Kananat\_result\lr scheduler exp\models"

train_models(model_to_train, train_data_dir, test_data_dir, results_base_dir, models_base_dir, max_epochs=30, batch_size=8)

Training ResNet152...
Loading training data from D:\Kananat\TF_TMJOA_jpg_x_5px...
Found 34203 files belonging to 2 classes.
Built model
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_7 (Dense)             (None, 512)               524800    
                                                       

In [ ]:
# model_name = 'EfficientNetB7'
# test_data_dir = r"D:\Kananat\TF_TMJOA_jpg_x_5px_test"

# models_base_dir = r"D:\Kananat\_result\lr scheduler exp\models"
# models_dir = os.path.join(models_base_dir, model_name)

# results_base_dir = r"D:\Kananat\_result\lr scheduler exp\logs"
# results_dir = os.path.join(results_base_dir, model_name)

# test_data = load_test_data(test_data_dir, model_name, batch_size=1)
# test_model(model_name, test_data, models_dir, results_dir)